
First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import os
import tensorflow as tf
import keras
import pandas as pd
import time 
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.optimizers import RMSprop
import nltk

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import random

from tensorflow.keras.callbacks import ModelCheckpoint  
sns.set_context()
sns.set_style()

def seed_everything(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    tf.random.set_seed(SEED)
    
seed_everything(6000)


Using TensorFlow backend.


## Hyper-parameter

In [2]:
BATCH_SIZE = 256
SEQ_LENGTH = 10
BUFFER_SIZE = 10000

## Load in Data


In [3]:

df = pd.read_csv('./data/nlp_finalproj_data_withids.csv').iloc[:,0:11]
# check any ducplicated r_id
unique_rid = set(df['r_id'])


# remove ducplicated r_id
df.drop_duplicates(subset = ['r_id'],keep = 'first',inplace = True)

df = df[df['cuisine']=='Chinese']
len(df)

243

In [4]:
#docs = [instruction for instruction in df['instructions']]
# concatenate all the instuctions together
docs = [str(instruction).replace('\n','').replace('\xa0','') for instruction in df['instructions']]
#text = "".join(str(instruction) for instruction in df['instructions'])
#text = text.replace('\n','').replace('\xa0','')



In [5]:
from nltk.tokenize.regexp import (
    RegexpTokenizer,
    WhitespaceTokenizer,
    BlanklineTokenizer,
    WordPunctTokenizer,
    wordpunct_tokenize,
    regexp_tokenize,
    blankline_tokenize,
)

In [6]:
text_token = [WordPunctTokenizer().tokenize(x.lower()) for x in docs]


In [7]:
stopwords = ['the','a','.']
corpus = [token for recipe in text_token for token in recipe]
corpus = [w for w in corpus if not w in stopwords] 
vocab = set(corpus)
len(vocab)

2573

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [8]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. idx2word
# 2. word2idx
word = sorted(tuple(vocab))


idx2word = dict((i, c) for i, c in enumerate(word))
idx2word[len(word)] = '<END>'
word2idx = {ww: ii for ii, ww in idx2word.items()}

# encode the text
encoded = np.array([word2idx[x] for x in corpus] + [len(word)])

In [9]:
idx2word

{0: '!',
 1: '!)',
 2: '!),',
 3: '!).',
 4: '!</',
 5: '"',
 6: '")',
 7: '".',
 8: '%',
 9: '&',
 10: "'",
 11: '(',
 12: ')',
 13: '),',
 14: ').',
 15: ');',
 16: '*',
 17: '**',
 18: ',',
 19: '-',
 20: '--',
 21: '."',
 22: '.)',
 23: '.).',
 24: '.*',
 25: '.**',
 26: '.-',
 27: '.</',
 28: '.•',
 29: '/',
 30: '0',
 31: '00',
 32: '000',
 33: '012345rating',
 34: '1',
 35: '10',
 36: '103',
 37: '109',
 38: '11',
 39: '12',
 40: '12minutes',
 41: '13',
 42: '13x9',
 43: '14',
 44: '140',
 45: '145',
 46: '15',
 47: '150',
 48: '153',
 49: '157',
 50: '16',
 51: '165',
 52: '165f',
 53: '17',
 54: '177c',
 55: '18',
 56: '180',
 57: '180c',
 58: '181',
 59: '19',
 60: '199',
 61: '1991',
 62: '1a',
 63: '1½',
 64: '2',
 65: '20',
 66: '2000',
 67: '2003',
 68: '2004',
 69: '2005',
 70: '2006',
 71: '2008',
 72: '2009',
 73: '200c',
 74: '2010',
 75: '2011',
 76: '2012',
 77: '2013',
 78: '2017',
 79: '20172016',
 80: '2017april',
 81: '2017author',
 82: '2017february',
 83: '201

And we can see those same characters from above, encoded as integers.

In [10]:
encoded[:100]

array([ 661, 1215, 1257, 1136, 1068, 1264,  234, 2280,  661,  642, 1181,
       2429, 2296, 1841,   11,  496,   14, 2149, 1215,  955,  152, 1385,
       1343,   18, 1096, 2421, 1472, 1136, 1241, 1894, 1510, 1353,   19,
       1306, 1096,  191,  997,  234,  585,   18, 2171,   18,  955,   64,
       1385,   18,  347,  467, 1449, 2324,  416,  997, 1742, 1096, 2324,
       1353,  234,  191, 2150, 1215,  234, 1909,  585,  955,   64, 1385,
        234, 2280,  191, 1843,  234,  585,  955,  237,   64, 1385,   18,
       2171,  191, 2087, 1893,  234,  585,  955,  105, 1935, 1404, 1096,
       2507, 1510, 1353, 1105, 1096,  191, 1472,  167, 2280,  191, 2375,
        234])

## Pre-processing the data


In [11]:
vocab_size = len(idx2word)
batch_size = 512

In [12]:
df.to_csv('../working/test.csv')

In [13]:
class NetworkDataGenerator(keras.utils.Sequence):
    """
    This is the class for construct data generator which build data batches for model training
    """

    def __init__(self, sentences, sequence, batch_size):
        """
        Initialize the data generator
        """

        self.sentences = sentences
        self.sequence = sequence
        self.batch_size = batch_size

    def __len__(self):
        """
        Denotes the number of batches per epoch
        """

        number_of_batches = int(np.ceil(len(self.sentences)) / self.batch_size)
        return number_of_batches

    def __getitem__(self, index):
        """
        Generate one batch of data

        :param index (int): The batch index for slicing
        :return tuple (batch_data, batch_label): Tuple of data and labels
        """
        sequence = 10

        sentences = []
        next_chars = []


        for i in range(0, len(corpus) - sequence):
 
            sentences.append(corpus[i: i + sequence])
            next_chars.append(corpus[i + sequence])


        number_of_sentences = len(sentences)
        starting_sentence_index = index * self.batch_size
        ending_sentence_index = ((index + 1) * (self.batch_size)) - self.sequence
        
        data = np.zeros((self.batch_size, self.sequence, vocab_size))
        labels = np.zeros((self.batch_size, vocab_size))
        
        for i, sentence in enumerate(self.sentences[starting_sentence_index : ending_sentence_index]):
            for t, char in enumerate(sentence):
                data[i, t, word2idx[char]] = 1
            labels[i, word2idx[next_chars[i]]] = 1
        
        return (data, labels)
        


In [14]:
def lstm(chars,sequence):
    sentences = []
    next_chars = []
    print('Creating sentences for model')
    progress = tqdm(total = len(corpus) - sequence)
    for i in range(0, len(corpus) - sequence):
        progress.update(1)
        sentences.append(corpus[i: i + sequence])
        next_chars.append(corpus[i + sequence])
    progress.close()
    number_of_sentences = len(sentences)
    
    print('Creating Generator')
    network_generator = NetworkDataGenerator(sentences = sentences, sequence= sequence, 
                                             batch_size = batch_size)
    
    model = Sequential()
    embedding_dim = 512
    #model.add(keras.layers.SimpleRNN(embedding_dim, input_shape=(sequence, vocab_size)))
    model.add(keras.layers.LSTM(embedding_dim, input_shape=(sequence, vocab_size)))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    
    return model, network_generator
    

In [15]:
def fit_model(filepath, model, network_generator):
    checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor = 'val_loss',
                                verbose = 0,
                                save_best_only = True,
                                save_weights_only = False,
                                mode = 'auto',
                                period = 1)
    
    callbacks_list = [checkpoint]


    start = time.time()
    history = model.fit_generator(generator = network_generator, steps_per_epoch = len(network_generator), 
                                  epochs=100, callbacks=callbacks_list)
    end = time.time()
    duration = end - start
    print("Training Time : %8.4f (s)" % (duration))

In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)





In [17]:
chinese_word_model, rnn_10_network_generator = lstm(idx2word,10)
chinese_word_model.summary()

100%|██████████| 38272/38272 [00:00<00:00, 484357.78it/s]

Creating sentences for model
Creating Generator


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               6322176   
_________________________________________________________________
dense_1 (Dense)              (None, 2574)              1320462   
_________________________________________________________________
activation_1 (Activation)    (None, 2574)              0         
Total params: 7,642,638
Trainable params: 7,642,638
Non-trainable params: 0
_________________________________________________________________


In [18]:
fit_model('chinese_word.hdf5',chinese_word_model, rnn_10_network_generator)


Epoch 1/100
74/74 [==============================] - 15s 202ms/step - loss: 5.0397
Epoch 2/100
 1/74 [..............................] - ETA: 11s - loss: 4.7146

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


74/74 [==============================] - 13s 179ms/step - loss: 4.7128
Epoch 3/100
74/74 [==============================] - 14s 191ms/step - loss: 4.8410
Epoch 4/100
74/74 [==============================] - 13s 181ms/step - loss: 4.6830
Epoch 5/100
74/74 [==============================] - 14s 185ms/step - loss: 4.6767
Epoch 6/100
74/74 [==============================] - 13s 182ms/step - loss: 4.5739
Epoch 7/100
74/74 [==============================] - 14s 185ms/step - loss: 4.3837
Epoch 8/100
 2/74 [..............................] - ETA: 17s - loss: 4.2067

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.203413). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


74/74 [==============================] - 13s 181ms/step - loss: 4.1804
Epoch 9/100
74/74 [==============================] - 14s 186ms/step - loss: 3.8992
Epoch 10/100
74/74 [==============================] - 14s 184ms/step - loss: 3.5855
Epoch 11/100
 2/74 [..............................] - ETA: 19s - loss: 3.1130

...

74/74 [==============================] - 14s 184ms/step - loss: 0.2130
Epoch 90/100
74/74 [==============================] - 14s 190ms/step - loss: 0.1657
Epoch 91/100
 2/74 [..............................] - ETA: 20s - loss: 0.0342

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.255947). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


74/74 [==============================] - 15s 196ms/step - loss: 0.1165
Epoch 92/100
74/74 [==============================] - 14s 186ms/step - loss: 0.1417
Epoch 93/100
74/74 [==============================] - 13s 179ms/step - loss: 0.1210
Epoch 94/100
74/74 [==============================] - 14s 185ms/step - loss: 0.1323
Epoch 95/100
74/74 [==============================] - 14s 183ms/step - loss: 0.1194
Epoch 96/100
74/74 [==============================] - 13s 181ms/step - loss: 0.1174
Epoch 97/100
74/74 [==============================] - 14s 183ms/step - loss: 0.1183
Epoch 98/100
74/74 [==============================] - 13s 182ms/step - loss: 0.1230
Epoch 99/100
74/74 [==============================] - 13s 182ms/step - loss: 0.1201
Epoch 100/100
 2/74 [..............................] - ETA: 17s - loss: 0.2008

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.204100). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


74/74 [==============================] - 13s 177ms/step - loss: 0.1118
Training Time : 1361.3669 (s)


In [19]:
#test_model = tf.keras.models.load_model("chinese_word.hdf5")

## Rebuild the model by checkpoint

In [20]:
# from keras.models import load_model
# # from tensorflow.keras.models import load_model
# model = tf.keras.models.load_model("/kaggle/working/chinese_word.hdf5")

In [21]:
def generate_text(model, length, diversity,sequence, starting_string):
    starting_string = WordPunctTokenizer().tokenize(starting_string.lower())
 
    sentence = starting_string[-10:]

    generated = ''
    generated = generated + ' '.join(sentence)
   
    for i in range(length):
        
            x_pred = np.zeros((1, sequence, len(vocab)+1))
    

            for t, char in enumerate(sentence):
                #print('sentence',sentence)
                
                x_pred[0, t-1, word2idx[char]] = 1.
     
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
    
            next_char = idx2word[next_index]


            generated = generated + ' '+ str(next_char) 
    
            sentence = sentence[1:] + [next_char]
            #print('next_char',next_char)
            #print('sentence', sentence)
    return generated


In [22]:
# [ 62 114 187  46  95]

seed_everything(62)
test_text = generate_text(chinese_word_model,100,0.2,10,starting_string="cooking until rice is heated through and peas are just cooked Serve with")
print(test_text)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


is heated through and peas are just cooked serve with for to , 2 strips 1 pepper pan : stirring cooked in size pat , cooked cilantro cook for stirring in add , dry large cooked and of nonstick large steam over high heat how sauce rice 1 oil and ingredients salt of steam - over - serve another , together : until ingredients salt of fry bright oils heat add if in add salt to 1 over or until toss of beef turn are rice sauté temperature for wok steak marinade , seconds cook for seconds more toss let rest then just room coat , immediately container , whisk


In [23]:
test_text

'is heated through and peas are just cooked serve with for to , 2 strips 1 pepper pan : stirring cooked in size pat , cooked cilantro cook for stirring in add , dry large cooked and of nonstick large steam over high heat how sauce rice 1 oil and ingredients salt of steam - over - serve another , together : until ingredients salt of fry bright oils heat add if in add salt to 1 over or until toss of beef turn are rice sauté temperature for wok steak marinade , seconds cook for seconds more toss let rest then just room coat , immediately container , whisk'